# Classificador Binário Multiclasse

### Importar dataset Mnist

In [ ]:
from sklearn.datasets import fetch_openml
import numpy as np

mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)
mnist.target = mnist.target.astype(np.int8)

X, y = mnist["data"], mnist["target"]

np.save('mnistX', X)
np.save('mnisty', y)

X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

### Classificador SVM - com SGD

#### Ajustando o modelo

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(loss='hinge', max_iter=5, tol=-np.infty, random_state=42)
sgd_clf.fit(X_train, y_train)

#### Verificações

In [ ]:
classe = 4
enum = (y_test == classe)
dig = np.where(enum == True)
digito = dig[0][1] # Alterar para ver as saídas
print(f"Dígito real: ", y_test[digito])

#### Resultados

In [ ]:
print(f"Classe Predita com SGDClassifier: ", sgd_clf.predict([X_test[digito]]))
some_digit_scores = sgd_clf.decision_function([X_test[digito]])
print(some_digit_scores)
print(f"Posição do vetor de maior valor (0 a 9): ", np.argmax(some_digit_scores))
print(f"Classe correspondente à posição do vetor: ", sgd_clf.classes_[sgd_clf.predict([X_test[digito]])]) # Altere o valor...
print("Score: ", sgd_clf.score(X_test, y_test))

#### Validação Cruzada

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring="accuracy")
print("Scores: ", scores)
print("%0.2f acurácia com um desvio padrão de %0.2f" % (scores.mean(), scores.std()))

#### Classificador SVM - Um contra Um - SGD

In [ ]:
from sklearn.multiclass import OneVsOneClassifier

ovo_clf = OneVsOneClassifier(SGDClassifier(loss='hinge', max_iter=5, tol=-np.infty, random_state=42))
ovo_clf.fit(X_train, y_train)
print(f"Classe Predita com OneVsOne Classifier: ", ovo_clf.predict([X_test[digito]]))
print(f"Quantidade de estimadores: ", len(ovo_clf.estimators_))
print("Score: ", ovo_clf.score(X_test, y_test))

#### Classificador SVM - Um contra todos - SGD

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

ova_clf = OneVsRestClassifier(SGDClassifier(loss='hinge', max_iter=5, tol=-np.infty, random_state=42))
ova_clf.fit(X_train, y_train)
print(f"Classe Predita com OneVsRest Classifier: ", ova_clf.predict([X_test[digito]]))
print(f"Quantidade de estimadores: ", len(ova_clf.estimators_))
print("Score: ", ova_clf.score(X_test, y_test))